In [23]:
# autoreload reloads modules automatically before entering the execution of code typed at the IPython prompt.
%reload_ext autoreload
%autoreload 2
%aimport feature_preprocessing_pipeline
%aimport build_model_by_brand
%aimport build_model

In [24]:
from sklearn import set_config
from os import system
from os.path import join, isfile
from joblib import dump, load
from feature_preprocessing_pipeline import categorize, extract_features, clean_variables, evaluate_prediction
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn import tree
from build_model_by_brand import evaluate_all_models
from build_model import get_one_model_for_all
#from dtreeviz.trees import dtreeviz

In [3]:
set_config(display='diagram') 

In [4]:
dataset_directory_path = 'dataset/'
model_directory_path = 'model/'

In [32]:
model_name = 'mercedes_optimize_rmse_2427'
model_filename = f'{model_name}.joblib'
model_path = join(model_directory_path, model_filename)
model = load(model_path)

In [60]:
def get_data(file_path, target_column,dataset_directory_path='./dataset/', sample=None, callback=None):
    data = pd.read_csv(join(dataset_directory_path, file_path), index_col=0)
    if callback:
        data = callback(data)
    if sample:
        data = data.sample(n=sample)
    X = data.drop(labels=[target_column], axis=1)
    y = np.log(data[target_column])
    return X,y



In [82]:
X_test = pd.read_csv(join(dataset_directory_path, 'X_test.csv'), index_col=0)
y_test = pd.read_csv(join(dataset_directory_path, 'y_test.csv'), index_col=0)

## Description du model 

### Diagramme

In [33]:
model

Pipeline(steps=[('features_extraction',
                 FunctionTransformer(func=<function extract_features at 0x7ffabad6dca0>,
                                     kw_args={'features': ['age',
                                                           'tax_per_year']})),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('poly', PolynomialFeatures(),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7ffab9321310>),
                                                 ('mpg_pipe',
                                                  Pip...
                                                                                               'Automatic',
                                                                                               'Semi-Auto']]))]),
                                                  ['transmission']),
                                                 ('fuel_type_pipe',
                                                  Pipeline(steps=[('OHE',
                                                                   OneHotEncoder(categories=[['Hybrid',
                                                                                              'Diesel',
                                                                                              'Petrol']],
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('OE',
                                                                   'passthrough')]),
                                                  ['fuel_type'])])),
                ('random_forest',
                 RandomForestRegressor(max_depth=50, max_features=None,
                                       min_samples_split=6, n_estimators=10,
                                       n_jobs=-1, verbose=False))])

In [6]:
model

Pipeline(steps=[('features_extraction',
                 FunctionTransformer(func=<function extract_features at 0x7ffabad6daf0>,
                                     kw_args={'features': ['age',
                                                           'tax_per_year']})),
                ('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('poly',
                                                  PolynomialFeatures(degree=1,
                                                                     interaction_only=True),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x...
                                                                                               'Semi-Auto',
                                                                                               'Automatic']]))]),
                                                  ['transmission']),
                                                 ('fuel_type_pipe',
                                                  Pipeline(steps=[('OHE',
                                                                   OneHotEncoder(categories=[['Petrol',
                                                                                              'Diesel',
                                                                                              'Hybrid']],
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('OE',
                                                                   'passthrough')]),
                                                  ['fuel_type'])])),
                ('random_forest',
                 RandomForestRegressor(max_depth=50, max_features=None,
                                       min_samples_split=6, n_estimators=10,
                                       n_jobs=-1, verbose=False))])

In [124]:
model.fit(X_test,y_test.values.ravel())
rf = model[3]#.estimators_[1]
#print(rf.estimators_[0].feature_importances_)

#_ = tree.plot_tree(rf.estimators_[0], filled=True)


Extract Features
[Pipeline]  (step 1 of 4) Processing features_extraction, total=   0.0s
[ColumnTransformer] .......... (1 of 9) Processing poly, total=   0.0s
[ColumnTransformer]  (2 of 9) Processing mpg_discretizer, total=   0.0s
[ColumnTransformer]  (3 of 9) Processing tax_discretizer, total=   0.0s
[ColumnTransformer]  (4 of 9) Processing engine_size_discretizer, total=   0.0s
[Pipeline] ........ (step 1 of 1) Processing discretize, total=   0.0s
[ColumnTransformer] ..... (5 of 9) Processing year_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer] .... (6 of 9) Processing model_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer] .... (7 of 9) Processing brand_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Processing OHE, total=   0.0s
[ColumnTransformer]  (8 of 9) Processing transmission_pipe, total=   0.0s
[Pipeline] ............... (step 1 of 1) Pro

/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/preprocessing/_discretization.py:220: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 0 are removed. Consider decreasing the number of bins.
  warnings.warn('Bins whose width are too small (i.e., <= '
/home/ghostbot/anaconda3/envs/tf_cpu/lib/python3.8/site-packages/sklearn/ensemble/_forest.py:369: UserWarning: Warm-start fitting without increasing n_estimators does not fit new trees.
  warn("Warm-start fitting without increasing n_estimators does not "


### Variables

In [121]:
rfr = model[3]
print(f"Le model {rfr} a {rfr.n_features_in_} variables")

Le model RandomForestRegressor(max_depth=50, min_samples_split=6, n_jobs=-1,
                      verbose=True, warm_start=True) a 256 variables


### Durée d'entraînement

## Performance

### Learning Curve

![Learning curve](./Learning_curve_model_677.png)

Le premier graphique montre la courbe d'apprentissage.

Le deuxième graphique montre le temps nécessaire aux modèles pour s'entraîner avec différentes tailles de données d'entraînement. 

Le troisième graphique montre le temps nécessaire à l'entraînement des modèles pour chaque taille de données d'entraînement.

In [64]:
print(f"Test set:\n X {X_test.shape} y {y_test.shape}")


Test set:
 X (7824, 9) y (7824, 1)


In [80]:
evaluate_prediction(model, X_test, y_test,sample=25)


Extract Features
RMSE: 1529.201266839476


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


MODEL PREDICTION TESTING ────────────────────────────────────────────────────────────────────

Prediction results                              
┏━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━┓
┃ Prediction ┃ Real Price ┃ Error ┃ Percentage ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━┩
│    7766    │    7341    │  425  │    6 %     │
│   15606    │   16000    │  394  │    2 %     │
│   11309    │   10997    │  312  │    3 %     │
│   19608    │   19279    │  329  │    2 %     │
│   11108    │   10999    │  109  │    1 %     │
│   19534    │   17894    │ 1640  │    9 %     │
│   14757    │   14488    │  269  │    2 %     │
│   42085    │   41630    │  455  │    1 %     │
│    6681    │    7499    │  818  │    11 %    │
│    7906    │    8995    │ 1089  │    12 %    │
│    6218    │    5998    │  220  │    4 %     │
│   13150    │   13494    │  344  │    3 %     │
│   10682    │   12500    │ 1818  │    15 %    │
│   14905    │   13599    │ 1306  │    10 %    │
│    8628    │    8740    │  112  │    1 %     │
│   38590    │   34950    │ 3640  │    10 %    │
│   11916    │   12700    │  784  │    6 %     │
│   12977    │   13494    │  517  │    4 %     │
│   10350    │   14591    │ 4241  │    29 %    │
│   20696    │   20500    │  196  │    1 %     │
│   16640    │   16497    │  143  │    1 %     │
│   41711    │   45490    │ 3779  │    8 %     │
│    8135    │    6994    │ 1141  │    16 %    │
│   16675    │   16297    │  378  │    2 %     │
│   22166    │   22095    │  71   │    0 %     │
└────────────┴────────────┴───────┴────────────┘

## Context d'application

Décrire le contexte d'évaluation du model. cdts dans lesquelles la précision est valable.

### Limites

Décrire les limites dans lesquelles le modèle est utilisable.

## Heuristique, analyse et choix

### Traitement du jeu de données

#### Suppression des valeurs extrêmes

##### Avant séparation du jeu de données

Suppression des valeurs extrêmes avant de scinder le jeu de données en ensemble d'entraînement, de validation et de test: 

Évaluation brute, RMSE: `1810.4777071283058`

##### Après séparation du jeu de données

Séquence:
1. Scission du jeu de données en *ensemble d'entraînement* et *ensemble de test*
1. Suppression des valeurs extrêmes dans l'*ensemble d'entraînement*

Évaluation brute, RMSE: `1879.4122621120684`


#### Imputation des valeurs extrêmes

##### Avant séparation du jeu de données

Séquence:
1. marquage des valeurs extrêmes
1. Imputation des valeurs marquées:
   * **KNNImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'ensemble des valeurs marquées
       * Évaluation brute, RMSE: `1863.9720250598702`
       
   * **IterativeImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'*ensemble d'entraînement* uniquement
       * Évaluation brute, RMSE: `1869.5612685708281`

##### Après séparation du jeu de données

Séquence:
1. Scission du jeu de données en *ensemble d'entraînement* et *ensemble de test*
1. Marquage des valeurs extrêmes dans l'*ensemble d'entraînement*
1. Imputation des valeurs marquées dans l'ensemble *ensemble d'entraînement*:
   * **KNNImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'*ensemble d'entraînement* uniquement
       * Évaluation brute, RMSE: `1885.9952078731876`
   * **IterativeImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'*ensemble d'entraînement* uniquement
       *  Évaluation brute, RMSE: `1848.2712109813936`
       
1. Scission de l'*ensemble d'entraînement* en un *ensemble de validation* et un d'*entraînement*

#### Suppression et imputation

séquence:
1. Suppression des valeurs extrêmes sur l'ensemble des données
1. Imputation des valeurs marquées dans l'ensemble *ensemble d'entraînement*:
   * **KNNImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'*ensemble d'entraînement* uniquement
       * Évaluation brute, RMSE: `1796.7700517404032`
   * **IterativeImputer**: 
       * Entraîné sur l'ensemble des valeurs des variables du jeu de données
       * Transformation de l'*ensemble d'entraînement* uniquement
       * Évaluation brute, RMSE: `1803.47188168163`
1. Scission de l'*ensemble d'entraînement* en un *ensemble de validation* et un d'*entraînement*

## Méthode

<center><img src="./assets/one_model_by_brand_overview.svg"/></center>

### Traitement des fichiers sources
<center><img src="./assets/file_processing.svg"/></center>

### Préparation des données

<center><img src="./assets/prepare_data.svg"/></center>

### Séparation des données

<center><img src="./assets/split_data.svg"/></center>

### Création du Modèle

<center><img src="./assets/build_model.svg"/></center>

#### Extract Features

Création de nouvelles variables:
1. `age`: $max(year)-year$
1. `model_count`: nombre d'occurence du modèle dans le dataset
1. `occurence_relative`: $\frac{count(model)}{count(allmodel)}$
1. `mileage_per_year`: $\frac{mileage}{age}$
1. `mpg_per_year`: $\frac{mpg}{age}$
1. `tax_per_year`: $\frac{tax}{age}$
1. `engine_size_per_year`: $\frac{enginesize}{age}$
1. `esoteric_composite_ratio`

Suivant la marque de voiture et/ou le modèle de prédiction créé, certaines améliorent le score de prédictions.

#### Transform Features
1. `PolynomialFeature` pour les variables numériques.
1. `KBinsDiscretizer` pour les variables `mpg`, `tax`, `engine_size`, `year`.
1. `OneHotEncoder` ou `OrdinalEncoder` pour les variables `model`, `brand`, `transmission`, `fuel_type`. 
   Pour l'`OrdinalEncoder`, les catégories ont été triées relativement à la variable dépendante `price` par ordre ascendant.


### Hyperparamètres

<center><img src="./assets/best_params_search.svg"/></center>

## Recherche de Modèles

### Par marques

Création d'un modèle par marques

In [19]:
# load performance
evaluate_all_models(search_term='model',exclude=['base_score_model.joblib'])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                 
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃ Target mean ┃ Prediction mean ┃          Model path           ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Toyota  │ 1018 │    12543    │      12424      │  ./model/toyota_model.joblib  │
│ Mercedes │ 2742 │    24606    │      24386      │ ./model/mercedes_model.joblib │
│   Audi   │ 2866 │    23179    │      23099      │   ./model/audi_model.joblib   │
│   Bmw    │ 2699 │    22720    │      22594      │   ./model/bmw_model.joblib    │
│   Ford   │ 1133 │    12214    │      12240      │   ./model/ford_model.joblib   │
│    Vw    │ 1294 │    16745    │      16690      │    ./model/vw_model.joblib    │
│   Opel   │ 1100 │    10471    │      10445      │   ./model/opel_model.joblib   │
│ Hyundai  │ 1262 │    12806    │      12847      │ ./model/hyundai_model.joblib  │
│  Skoda   │ 1300 │    14096    │      14077      │  ./model/skoda_model.joblib   │
└──────────┴──────┴─────────────┴─────────────────┴───────────────────────────────┘

In [20]:
# load performance
evaluate_all_models(search_term='optimize',exclude=['base_score_model.joblib'])

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                    
┏━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand   ┃ RMSE ┃ Target mean ┃ Prediction mean ┃            Model path            ┃
┡━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Mercedes │ 3961 │    24606    │      24297      │ ./model/mercedes_optimize.joblib │
│  Toyota  │ 1630 │    12543    │      12354      │  ./model/toyota_optimize.joblib  │
│   Bmw    │ 3991 │    22720    │      22494      │   ./model/bmw_optimize.joblib    │
│    Vw    │ 2110 │    16745    │      16708      │    ./model/vw_optimize.joblib    │
│   Opel   │ 1179 │    10471    │      10402      │   ./model/opel_optimize.joblib   │
│   Ford   │ 1338 │    12214    │      12130      │   ./model/ford_optimize.joblib   │
│  Skoda   │ 1776 │    14096    │      14128      │  ./model/skoda_optimize.joblib   │
│ Hyundai  │ 1178 │    12806    │      12711      │ ./model/hyundai_optimize.joblib  │
│   Audi   │ 1847 │    23179    │      23057      │   ./model/audi_optimize.joblib   │
└──────────┴──────┴─────────────┴─────────────────┴──────────────────────────────────┘

In [31]:
# load performance
evaluate_all_models(search_term='rmse')

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                           
┏━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Brand  ┃ RMSE ┃ Target me… ┃ Prediction mean ┃                Model path                 ┃
┡━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│  Opel   │ 1191 │   10471    │      10386      │   ./model/opel_optimize_rmse_679.joblib   │
│ Hyundai │ 1178 │   12806    │      12711      │ ./model/hyundai_optimize_rmse_733.joblib  │
│  Ford   │ 1338 │   12214    │      12130      │   ./model/ford_optimize_rmse_801.joblib   │
│   Bmw   │ 3991 │   22720    │      22494      │   ./model/bmw_optimize_rmse_1635.joblib   │
│ Toyota  │ 1630 │   12543    │      12354      │  ./model/toyota_optimize_rmse_919.joblib  │
│   Vw    │ 2110 │   16745    │      16708      │   ./model/vw_optimize_rmse_1162.joblib    │
│ Merced… │ 3894 │   24606    │      24298      │ ./model/mercedes_optimize_rmse_2427.jobl… │
│  Audi   │ 1842 │   23179    │      23047      │  ./model/audi_optimize_rmse_1842.joblib   │
│  Skoda  │ 1731 │   14096    │      14089      │  ./model/skoda_optimize_rmse_1428.joblib  │
└─────────┴──────┴────────────┴─────────────────┴───────────────────────────────────────────┘

### Toutes marques

1. fusion des données en une seule source de données

In [ ]:
get_

### Entraînement Incrémental

Le model est entrainé sur chaque jeu de données en conservant les arbres précédemment créés.

In [30]:
get_one_model_for_all(model_to_dump=True, evaluation=True, verbose=False)

PERFORMANCE COMPARISON ──────────────────────────────────────────────────────────────────────

One model by Brand                                                                         
┏━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Brand    ┃ RMSE ┃ Target mean ┃ Prediction mean ┃             Model path              ┃
┡━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inc. Model │ 8652 │    16802    │      11661      │ model/all_brand_model_8652.0.joblib │
└────────────┴──────┴─────────────┴─────────────────┴─────────────────────────────────────┘